## CNN

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/cats-vs-dogs/PetImages",
    seed=123,
    image_size=(180, 180),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/cats-vs-dogs/PetImages",
    seed=123,
    image_size=(180, 180),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    shuffle=True
)

# Step to take an image from the dataset
# =============================================
# for images, labels in train_ds.take(1):
#     image = images[0].numpy().astype("uint8")
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()
# =============================================

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(180,180,3)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(180,180,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=[early_stop]
)

test_loss, test_acc = model.evaluate(test_ds)
print(f'Test accuracy: {test_acc}')

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Model Accuracy - {test_acc*100}%')
plt.legend()
plt.show()

model.save("cat_vs_dog_ImageClassification.keras")